In [1]:
import numpy as np
import mne
import matplotlib.pyplot as plt
import os
from numpy import diff
import statistics 

In [2]:
#Takes period segment, voltage threshold, min spike duration in seconds, max spike duration in seconds, and the sampling frequency, returns the amount of spikes in the segment udner those conditions.
def spikeamount_from_Mean(segment, spike_thresh, spike_min, spike_max, samp_freq):
    sample_count = 0
    spikes = 0
    spike_min = spike_min*samp_freq
    spike_max = spike_max*samp_freq
    time_of_spike = []
    Mean_value = statistics.mean(segment)
    
    for sample in segment:
        if sample > spike_thresh+Mean_value:
            sample_count += 1
            
        elif sample_count > spike_min and sample_count < spike_max:
            spikes += 1
            sample_count = 0
        else:
            sample_count = 0
    return spikes

def spikeamount(segment, spike_thresh, spike_min, spike_max, samp_freq):
    sample_count = 0
    spikes = 0
    spike_min = spike_min*samp_freq
    spike_max = spike_max*samp_freq
    time_of_spike = []
        
    for sample in segment:
        if sample > spike_thresh:
            sample_count += 1
            
        elif sample_count > spike_min and sample_count < spike_max:
            spikes += 1
            sample_count = 0
        else:
            sample_count = 0
    return spikes


#Takes a period as list of samples, desired segment length in seconds and the sampling frequency, returns a list of segments of the period {{},{},...{}}
def splitperiod(period, segment_length, sampling_freq):
    segments = []
    current_segment = []
    sample_seglen = segment_length * sampling_freq
    for i, sample in enumerate(period):
        current_segment.append(sample)
        if i%sample_seglen == 0 and i!=0:
            segments.append(current_segment)
            current_segment = []
    return segments

#takes list of spike amounts, returns list of smoothed spike amounts (smoothed spike amount is the average of M neighbour spike amount)
def smoothspikes(spikes, M):
    smoothspikes = []
    window_min = -(M//2)
    window_max = M//2
    for spike in spikes:
        curravg = 0
        if window_min<0:
            for i in range(0,M):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        elif window_max>=len(spikes):
            for i in range (0,M):
                curravg+=spikes[len(spikes)-1-i]
            smoothspikes.append(curravg/M)
        else:
            for i in range(window_min, window_max+1):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        window_min +=1
        window_max +=1
    return smoothspikes

In [8]:
# Meant to be run separately from the warning loop
# calculates the threshold for the patient, set the data loaded as a file whitout seizures
# set this as your maxspikes_inter, 
data = mne.io.read_raw_edf('Data_chb03/chb03_05.edf').get_data()

raw_data = data
channels = []
for channel in raw_data:
   channels.append(splitperiod(channel,5,256))
spikes = []
for i, channel in enumerate(channels):
    spikes.append([])
    for segment in channel:
        spikes[i].append(spikeamount(segment, 0.0001,0.04,0.07,256))
smoothedspikes = []
for channel in spikes:
    smoothedspikes.append(smoothspikes(channel,7))

temp = []
for channel in smoothedspikes:
    temp.append(max(channel))
spikeMax = max(temp)
print(spikeMax)


3.7142857142857144


In [7]:

data = mne.io.read_raw_edf('Data_chb03/chb03_05.edf').get_data()

channels = []
for channel in raw_data:
   channels.append(splitperiod(channel,5,256))
spikes = []
for i, channel in enumerate(channels):
    spikes.append([])
    for segment in channel:
        spikes[i].append(spikeamount(segment, 0.0001,0.04,0.07,256))
smoothedspikes = []
for channel in spikes:
    smoothedspikes.append(smoothspikes(channel,7))

print(max(max(smoothedspikes)))

maxspikes_inter = 0.857142857
alarms = []
for channel in smoothedspikes:
    isdumped = 0
    for i,spikes in enumerate(channel):
        if spikes>maxspikes_inter*1.3:
            if isdumped == 0:
                alarms.append(i*5)
                isdumped = 1
            #print("Alarm at"+str(i*5)+"seconds")
print(alarms)

2.7142857142857144
Alarm at1065seconds
Alarm at1070seconds
Alarm at1075seconds
Alarm at1080seconds
Alarm at1085seconds
Alarm at1090seconds
Alarm at1310seconds
Alarm at1315seconds
Alarm at1320seconds
Alarm at1325seconds
Alarm at1330seconds
Alarm at1455seconds
Alarm at1460seconds
Alarm at1465seconds
Alarm at1470seconds
Alarm at1475seconds
Alarm at1480seconds
Alarm at1485seconds
Alarm at1570seconds
Alarm at1575seconds
Alarm at1580seconds
Alarm at1585seconds
Alarm at1590seconds
Alarm at1720seconds
Alarm at1725seconds
Alarm at1730seconds
Alarm at1735seconds
Alarm at3000seconds
Alarm at3005seconds
Alarm at3010seconds
Alarm at3015seconds
Alarm at3020seconds
Alarm at3025seconds
Alarm at3005seconds
Alarm at3010seconds
Alarm at3015seconds
Alarm at3020seconds
Alarm at3025seconds
Alarm at3030seconds
Alarm at3035seconds
Alarm at785seconds
Alarm at790seconds
Alarm at795seconds
Alarm at800seconds
Alarm at805seconds
Alarm at815seconds
Alarm at820seconds
Alarm at1350seconds
Alarm at600seconds
Alarm at6